## Boston Crimes Database Project

In this project, we build a PostgreSQL database to store and manage data on crimes committed in Boston.
The dataset (`boston.csv`) contains details such as crime identifiers, offense codes, descriptions, dates, days of the week, and geographic coordinates (latitude and longitude).

Our objectives:
- Create a database named **`crimes_db`** with a schema **`crimes`**.
- Define a table **`boston_crimes`** with appropriate datatypes for all columns.
- Set up user groups **`readonly`** and **`readwrite`** with corresponding privileges.
- Create one user per group for controlled database access.

This project brings together core data engineering concepts — schema design, access control, and structured data storage — to simulate a realistic end-to-end workflow.


In [4]:
import psycopg2

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="newpassword",
    host="localhost",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor()

In [5]:
cur.execute("CREATE DATABASE crime_db;")

cur.close()
conn.close()

In [7]:
conn = psycopg2.connect(dbname="crime_db", user="postgres")
cur = conn.cursor()

cur.execute("CREATE SCHEMA crimes;")
conn.commit()

In [9]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [10]:
def get_col_set(csv_filename, col_index):
    unique_values = set()
    with open(csv_filename) as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            unique_values.add(row[col_index])
    return unique_values

In [11]:
for i, name in enumerate(col_headers):
    unique_count = len(get_col_set("boston.csv", i))
    print(f"{name}: {unique_count} unique values")

incident_number: 298329 unique values
offense_code: 219 unique values
description: 239 unique values
date: 1177 unique values
day_of_the_week: 7 unique values
lat: 18177 unique values
long: 18177 unique values


In [12]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [14]:
descriptions = get_col_set("boston.csv", 2)

max_len = max(len(desc) for desc in descriptions)
print("Maximum description length:", max_len)

Maximum description length: 58


In [26]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [89]:
conn.rollback()

cur.execute("""
    CREATE TYPE day_enum AS ENUM (
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
    );
""")

In [90]:
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
    incident_number INT PRIMARY KEY,
    offense_code INT,
    description VARCHAR(100),
    date DATE,
    day_of_the_week day_enum,
    lat DECIMAL(10,8),
    long DECIMAL(11,8)
);

""")

In [91]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [92]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

In [93]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [94]:
cur.execute("CREATE GROUP readonly NOLOGIN;")

In [95]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")

In [96]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")

In [97]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

In [98]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")

In [99]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")


In [100]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [101]:
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

In [102]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

In [103]:
cur.execute("GRANT readonly TO data_analyst;")

In [104]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

In [105]:
cur.execute("GRANT readwrite TO data_scientist;")

In [109]:
cur.execute("""
SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';
""")

priveleges = cur.fetchall()

for row in priveleges:
    print(row)

('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


In [110]:
cur.execute("""
SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';
""")

priveleges = cur.fetchall()

for row in priveleges:
    print(row)

('readonly', 'SELECT')


In [111]:
cur.close()
conn.close()